In [1]:
import pandas as pd
import numpy as np
from progressbar import ProgressBar
import time
import math
import itertools
import ipdb
from tqdm import tqdm
import pickle

In [30]:
data = pd.read_csv('data/cities.csv', index_col='CityId')
output = pd.read_csv('submission.csv')

In [132]:
bar = ProgressBar()

In [2]:
def is_prime(n):
    if n == 1:
        return False
    i = 2
    while i*i <= n:
        if n % i == 0:
            return False
        i += 1
    return True

In [3]:
def getDistance():
    total_distance = 0
    from_index = None
    step = 1
    for i in bar(output.values):
        i = i[0]
        if from_index == None:
            from_index = i
            print(from_index)
        else:
            to_index = i
            distance = np.linalg.norm(np.array(data.loc[from_index]) - np.array(data.loc[to_index]))

            if step%10 == 0 and not is_prime(to_index):
                distance = distance * 1.1
            total_distance += distance
            from_index = to_index
        step += 1
    return total_distance

In [4]:
def get_score(output):
    plist = [i for i in range(len(output)) if is_prime(i)]
    cities = pd.read_csv('data/cities.csv')
    all_ids = cities['CityId'].values
    all_x = cities['X'].values
    all_y = cities['Y'].values

    arr = dict()
    for i, id in enumerate(all_ids):
        arr[id] = (all_x[i], all_y[i])

    score = 0.0
    s = output
    for i in tqdm(range(0, len(s)-1)):
        p1 = arr[s[i]]
        p2 = arr[s[i+1]]
        stepSize = math.sqrt((p1[0] - p2[0]) * (p1[0] - p2[0]) + (p1[1] - p2[1]) * (p1[1] - p2[1]))
        if ((i + 1) % 10 == 0) and (s[i] not in plist):
            stepSize *= 1.1
        # print(stepSize)
        score += stepSize
    return score

In [7]:
def getstep(window_size, idx):
    indices = [i+idx for i in range(window_size)]
    step = 10 - idx%10
    return step

In [8]:
def getNewDistance(ids, window_size, idx):
    if len(ids) == window_size:
        total_distance = 0
        from_index = None
        primeStep = getstep(window_size, idx)
        step = 1
        for i in ids:
            if from_index == None:
                from_index = i
            else:
                to_index = i
                distance = np.linalg.norm(np.array(data.loc[from_index]) - np.array(data.loc[to_index]))

                if step == primeStep and not is_prime(to_index):
                    distance = distance * 1.1
                total_distance += distance
                from_index = to_index
                step += 1
        return total_distance
    return None

In [9]:
def getFastDistance(ids, window_size, idx):
    vector1 = data.loc[ids]
    vector2 = vector1.shift(-1)[:-1]
    vector1 = vector1[:-1]
    distance = np.linalg.norm(vector1.values - vector2.values, axis=1)
    primeStep = getstep(window_size, idx)
    if primeStep !=0 and primeStep < window_size and is_prime(ids[primeStep]):
        distance[primeStep-1] *= 1.1
    distance = np.sum(distance)
    return distance

In [10]:
distance_measure_time = 0
def getCityIdsPermutation(p, ids, window_size, idx):
    #global distance_measure_time
    new_ids = []
    for i in p:
        new_ids.append(ids[i])
    #start = time.time()
    distance = getFastDistance(new_ids, window_size, idx)
    #distance_measure_time += time.time() - start
    return new_ids, distance

In [11]:
def arrangeCityIds(ids, window_size, idx):
    arrangeIds = []
    listCombination = []
    window = list(range(window_size))
    if len(ids) == window_size:
        #start = time.time()
        permutations = [[0]+list(i)+[window_size-1] for i in list(itertools.permutations(window[1:-1]))]
        for p in permutations:
            new_ids, distance = getCityIdsPermutation(p, ids, window_size, idx)
            listCombination.append((new_ids, distance))
        listCombination = sorted(listCombination,key=lambda d: d[1])
        #print('getCityIdsPermutation: ', time.time() - start)
        #print('DistanceMeasureTime: ', distance_measure_time)
        return listCombination[0][0]
    return ids

In [12]:
def primeArrange():
    data_len = len(new_output)
    i = 0
    window_size = 7
    #lower_index = int(window_size/2)
    end_index = data_len - window_size
    for id in bar(new_output):
        #if (i+1)%10 == 0 and i+2 < data_len:
        if i <= end_index:
            ids = new_output[i:i+window_size]
            #isPrime, primeIdx = oneSidePrimeCheck(ids)
            #start = time.time()
            new_output[i:i+window_size] = arrangeCityIds(ids, window_size, i)
            #print('arrangeCityIds: ', time.time() - start)
        i += 1

In [27]:
new_output = [i[0] for i in output.values]
version = 1
while True:
    previous_output = new_output
    ipdb.set_trace()
    primeArrange()
    pd.DataFrame({'Path': new_output}).to_csv('submission_ver_'+str(version)+'.csv', index=False)
    count = 0
    for i, cityId in enumerate(new_output):
        if cityId != previous_output[i]:
            count += 1
    if count == 0:
        break

None
> <ipython-input-27-b86d9c601114>(6)<module>()
      5     ipdb.set_trace()
----> 6     primeArrange()
      7     pd.DataFrame({'Path': new_output}).to_csv('submission_ver_'+str(version)+'.csv', index=False)

ipdb> s
--Call--
> <ipython-input-12-e789c0178ca1>(1)primeArrange()
----> 1 def primeArrange():
      2     data_len = len(new_output)
      3     i = 0

ipdb> n
> <ipython-input-12-e789c0178ca1>(2)primeArrange()
      1 def primeArrange():
----> 2     data_len = len(new_output)
      3     i = 0

ipdb> n
> <ipython-input-12-e789c0178ca1>(3)primeArrange()
      2     data_len = len(new_output)
----> 3     i = 0
      4     window_size = 7

ipdb> n
> <ipython-input-12-e789c0178ca1>(4)primeArrange()
      3     i = 0
----> 4     window_size = 7
      5     #lower_index = int(window_size/2)

ipdb> n
> <ipython-input-12-e789c0178ca1>(6)primeArrange()
      5     #lower_index = int(window_size/2)
----> 6     end_index = data_len - window_size
      7     for id in bar(new_output)

N/A% (0 of 197770) |                     | Elapsed Time: 0:00:00 ETA:  --:--:--

> <ipython-input-12-e789c0178ca1>(9)primeArrange()
      8         #if (i+1)%10 == 0 and i+2 < data_len:
----> 9         if i <= end_index:
     10             ids = new_output[i:i+window_size]

ipdb> n
> <ipython-input-12-e789c0178ca1>(10)primeArrange()
      9         if i <= end_index:
---> 10             ids = new_output[i:i+window_size]
     11             #isPrime, primeIdx = oneSidePrimeCheck(ids)

ipdb> n
> <ipython-input-12-e789c0178ca1>(13)primeArrange()
     12             #start = time.time()
---> 13             new_output[i:i+window_size] = arrangeCityIds(ids, window_size, i)
     14             #print('arrangeCityIds: ', time.time() - start)

ipdb> ids
[0, 47239, 177242, 167366, 38941, 70519, 114579]
ipdb> n
> <ipython-input-12-e789c0178ca1>(15)primeArrange()
     13             new_output[i:i+window_size] = arrangeCityIds(ids, window_size, i)
     14             #print('arrangeCityIds: ', time.time() - start)
---> 15         i += 1

ipdb> [i[0] for i in output.values][:7

  0% (36 of 197770) |           | Elapsed Time: 0:02:51 ETA:  10 days, 21:29:22


Program interrupted. (Use 'cont' to resume).
> /usr/local/lib/python3.6/dist-packages/pandas/core/internals.py(3368)_rebuild_blknos_and_blklocs()
   3367         new_blknos = np.empty(self.shape[0], dtype=np.int64)
-> 3368         new_blklocs = np.empty(self.shape[0], dtype=np.int64)
   3369         new_blknos.fill(-1)

ipdb> c


  0% (53 of 197770) |            | Elapsed Time: 0:02:56 ETA:  7 days, 15:06:50


Program interrupted. (Use 'cont' to resume).
--Call--
> /usr/local/lib/python3.6/dist-packages/pandas/core/dtypes/common.py(477)is_interval_dtype()
    476 
--> 477 def is_interval_dtype(arr_or_dtype):
    478     """

--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
--KeyboardInterrupt--
ipdb> exit


BdbQuit: 

In [17]:
get_score(np.array(new_output))

100% (197769 of 197769) |#################| Elapsed Time: 0:01:29 Time: 0:01:29


1533158.1934586992

In [41]:
l1 = [1,2,3]
l2 = [1,3,2]

In [33]:
import operator

In [42]:
operator.eq(l1,l2)

False

In [121]:
data_dict = {i:tuple(data.loc[i].values) for i in [55,56]}

In [122]:
data_dict

{55: (301.141495982181, 2794.8486758027802),
 56: (4980.96025926499, 2716.3960512961603)}

In [123]:
ids = np.array([[55,56],[56,55]])

In [124]:
f = lambda i: data_dict[i]
mat1 = np.vectorize(f)(ids)
#mat2 = mat1[:,1:]
#mat1 = mat1[:,:-1]

In [125]:
np.array(mat1)

array([[[ 301.14149598, 4980.96025926],
        [4980.96025926,  301.14149598]],

       [[2794.8486758 , 2716.3960513 ],
        [2716.3960513 , 2794.8486758 ]]])

In [69]:
mat1[0]

array([[ 301.14149598, 4980.96025926],
       [4980.96025926,  301.14149598]])

In [70]:
mat1[1]

array([[2794.8486758, 2716.3960513],
       [2716.3960513, 2794.8486758]])

In [85]:
d3 = np.array([mat1[0],mat1[1]])

In [90]:
d3s = np.array([mat1[0]*2,mat1[1]*2])

In [91]:
d3

array([[[ 301.14149598, 4980.96025926],
        [4980.96025926,  301.14149598]],

       [[2794.8486758 , 2716.3960513 ],
        [2716.3960513 , 2794.8486758 ]]])

In [92]:
d3s

array([[[ 602.28299196, 9961.92051853],
        [9961.92051853,  602.28299196]],

       [[5589.69735161, 5432.79210259],
        [5432.79210259, 5589.69735161]]])

In [93]:
d3 - d3s

array([[[ -301.14149598, -4980.96025926],
        [-4980.96025926,  -301.14149598]],

       [[-2794.8486758 , -2716.3960513 ],
        [-2716.3960513 , -2794.8486758 ]]])

In [97]:
np.linalg.norm(d3 - d3s, axis=2)

array([[4990.05524067, 4990.05524067],
       [3897.43333851, 3897.43333851]])

In [150]:
m = np.arange(27).reshape(3,3,3)

In [151]:
m

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [155]:
np.linalg.norm(m, axis=0)

array([[20.1246118 , 21.49418526, 22.91287847],
       [24.37211521, 25.86503431, 27.38612788],
       [28.93095228, 30.49590136, 32.07802986]])

In [109]:
m

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [113]:
m[:,:,1:]

array([[[ 0,  1],
        [ 3,  4],
        [ 6,  7]],

       [[ 9, 10],
        [12, 13],
        [15, 16]],

       [[18, 19],
        [21, 22],
        [24, 25]]])

In [11]:
output_v5 = pd.read_csv('submission_ver_i5_opt_2.csv')

In [12]:
get_score(output_v5.values.reshape(-1))

100%|██████████| 197769/197769 [01:27<00:00, 2267.86it/s]


1532991.8143511026

In [7]:
output_vi5 = pd.read_csv('submission_ver_i5_opt_1.csv')

In [8]:
get_score(output_vi5.values.reshape(-1))

100%|██████████| 197769/197769 [01:25<00:00, 2246.00it/s]


1532993.792467967

In [9]:
ov5 = output_vi5.values.reshape(-1)

In [10]:
s = [i for i,v in enumerate(output_v5.values.reshape(-1)) if ov5[i] != v]

In [14]:
output_v5.values.reshape(-1)[s[0]]

152827

In [15]:
ov5[s[0]]

132933

In [16]:
s[:5]

[437, 438, 1180, 1181, 1235]

In [17]:
with open('new_output.pickle', 'rb') as f:
    new_output = pickle.load(f)

In [11]:
with open('iteration.pickle', 'rb') as f:
    i = pickle.load(f)

In [25]:
d = np.zeros(shape=(5,5))

In [26]:
d

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [28]:
for r in range(d.shape[0]):
    for c in range(r+1,d.shape[0]):
        d[r][c] = r+c

In [32]:
data.loc[1].values

array([4377.40597217,  336.60208217])

In [33]:
data.loc[0].values

array([ 316.83673906, 2202.34070734])

In [34]:
data.loc[0].values -data.loc[1].values

array([-4060.5692331 ,  1865.73862516])

In [40]:
np.array(list(itertools.permutations([1,2,3])))

array([[1, 2, 3],
       [1, 3, 2],
       [2, 1, 3],
       [2, 3, 1],
       [3, 1, 2],
       [3, 2, 1]])

In [41]:
bin(10)

'0b1010'

In [71]:
inputvalue = 'f'
result = int(inputvalue, 16)
result

15

In [72]:
def baseN(num,b,numerals="0123456789abcdefghijklmnopqrstuvwxyz"):
    return ((num == 0) and numerals[0]) or (baseN(num // b, b, numerals).lstrip(numerals[0]) + numerals[num % b])

In [75]:
baseN(1000,16)

'3e8'

In [79]:
import string

In [81]:
string.digits + string.ascii_letters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [84]:
numerals = string.digits + string.ascii_letters

In [111]:
def getDistanceValue(r,c,ids):
    r_id = ids[r]
    c_id = ids[c]
    r_cord = data.loc[r_id].values
    c_cord = data.loc[c_id].values
    distance = np.linalg.norm(r_cord - c_cord)
    return distance


def getDistanceMatrix(ids):
    ids_len = len(ids)
    matrix = np.zeros(shape=(ids_len, ids_len))
    for r in range(ids_len):
        for c in range(r + 1, ids_len):
            matrix[r][c] = getDistanceValue(r,c,ids)
    return matrix

def baseN(num,b, numerals):
    return ((num == 0) and numerals[0]) or (baseN(num // b, b, numerals).lstrip(numerals[0]) + numerals[num % b])

def getPermuations(ids, nearest, window_size):
    disMatrix = getDistanceMatrix(ids)
    minDistaceIndex = list(range(nearest))
    base = window_size - 2
    minDistaceIndex = [baseN((i**2)*base, base, numerals) for i in minDistaceIndex]
    minDistaceIndex = [list(i) if len(i) == base else list('0'*(base-len(i)) + i) for i in minDistaceIndex]
    
    for i, indices in enumerate(minDistaceIndex):
        for j, idx in enumerate(indices):
            minDistaceIndex[i][j] = int(idx,base) + 1
    return minDistaceIndex

In [113]:
getPermuations([1,2,3], nearest=5, window_size=6)

[[1, 1, 1, 1], [1, 1, 2, 1], [1, 2, 1, 1], [1, 3, 2, 1], [2, 1, 1, 1]]

In [101]:
r = np.random.randint(0,100,size=(5,5))
r

array([[30, 63, 63, 27, 70],
       [48, 56,  6, 18, 70],
       [58, 44, 61, 32, 25],
       [25,  4, 75, 69, 52],
       [55, 44,  8, 20,  5]])

In [138]:
int('110',4)

20

In [115]:
baseN('110',4, numerals)

TypeError: unsupported operand type(s) for //: 'str' and 'int'

In [167]:
j = [i for i in range(10) if i%4 == 0]

In [168]:
j

[0, 4, 8]

In [242]:
import math

In [185]:
a=np.array([[0],[1],[2],[3]])
a=a*10
a

array([[ 0],
       [10],
       [20],
       [30]])

In [186]:
b=np.array([[0],[1],[2]])
b=np.transpose(b)
b

array([[0, 1, 2]])

In [188]:
c = (a+b).reshape(-1)

In [189]:
c

array([ 0,  1,  2, 10, 11, 12, 20, 21, 22, 30, 31, 32])

In [259]:
a = np.array([1,2,4,4])

array([[1],
       [2],
       [4],
       [4]])

In [334]:
a = np.array(list(range(12)))
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [335]:
dic = {i:a[:-(i+1)] for i in a}

In [336]:
dic[4]

array([0, 1, 2, 3, 4, 5, 6])

In [337]:
for i in range(0,len(dic)-1):
    a = a.reshape((np.prod(a.shape),1))*10 + dic[i]

MemoryError: 

In [ ]:
a.shape

In [20]:
get_score([124531, 138318, 173044, 57092, 40216, 152827, 132933, 169228, 151929, 40660, 172270])

100%|██████████| 10/10 [00:00<00:00, 29086.71it/s]


78.78906480549699

In [21]:
get_score([124531, 138318, 173044, 57092, 40216, 132933, 152827, 169228, 151929, 40660, 172270])

100%|██████████| 10/10 [00:00<00:00, 30885.89it/s]


78.88409663215559

In [338]:
a = ['3','4']

In [339]:
list(map(int, a))

[3, 4]

In [25]:
a = np.array([[1,3,4],[1,3,4],[1,3,4]])

In [27]:
a[0] = [4,5,3]

In [28]:
a

array([[4, 5, 3],
       [1, 3, 4],
       [1, 3, 4]])